In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
import telebot
from telebot import types

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import openpyxl

In [ ]:
headers =  {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
l = ["", "PAGEN_2=2&", "PAGEN_2=3&"]

def parsing():

  for j in range(3):

    sleep(3)

    url = f"https://guitar-guitar.ru/?q=%D1%81%D1%82%D1%83%D0%B4%D0%B8%D0%B9%D0%BD%D1%8B%D0%B9%20%D0%BA%D0%BE%D0%BD%D0%B4%D0%B5%D0%BD%D1%81%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9%20%D0%BC%D0%B8%D0%BA%D1%80%D0%BE%D1%84%D0%BE%D0%BD&s=&{l[j]}SIZEN_2=15"
    response = requests.get(url, headers = headers)
    soup = BeautifulSoup(response.text, "lxml")
    data = soup.find_all("div", class_ = "js-product-container product-list-item")

    for i in data:

      name = i.find("a", class_ = "product-name").text.replace("\t", "").replace("\n", "")
      price = i.find("div", class_ ="bx_price").text.replace("\t", "").replace("\n", "")
      url_img = "https://guitar-guitar.ru" + i.find("img", class_ = "lazy").get("data-src")

      yield name, price, url_img

In [ ]:
def record(parsing):

  book = openpyxl.Workbook()
  book.remove(book.active)
  sheet_1 = book.create_sheet('Студийный конденсаторный микрофон')

  sheet_1.column_dimensions['A'].width = 100
  sheet_1.column_dimensions['B'].width = 10
  sheet_1.column_dimensions['C'].width = 100

  row = 1
  column = 1

  for item in parsing():

    sheet_1.cell(row, column , item[0])
    sheet_1.cell(row, column+1 , item[1])
    sheet_1.cell(row, column+2,).hyperlink = item[2]

    row+=1

  book.save("/content/drive/MyDrive/АД и МО/Lab_5_6/Гитар-гитар.xlsx")
  book.close()

In [ ]:
bot = telebot.TeleBot('6392047296:AAFcVMaAmq3-unMLnSHTYoCE8UQOKNJHJ_k')

def on_click(message):

  del_bttn = telebot.types.ReplyKeyboardRemove()

  if message.text == 'Перейти на сайт':

    bot.send_message(message.chat.id, 'https://guitar-guitar.ru/?q=%D1%81%D1%82%D1%83%D0%B4%D0%B8%D0%B9%D0%BD%D1%8B%D0%B9%20%D0%BA%D0%BE%D0%BD%D0%B4%D0%B5%D0%BD%D1%81%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9%20%D0%BC%D0%B8%D0%BA%D1%80%D0%BE%D1%84%D0%BE%D0%BD&s=&SIZEN_2=15')
    bot.register_next_step_handler(message, on_click)

  elif message.text == 'Выполнить парсинг':

    bot.send_message(message.chat.id, 'Парсинг выполняется...',  reply_markup = del_bttn)
    record(parsing)
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    bttn_1 = types.InlineKeyboardButton('Перейти на сайт')
    bttn_2 = types.InlineKeyboardButton('Выполнить парсинг')
    bttn_3 = types.InlineKeyboardButton('Скачать .xlsx')
    bttn_4 = types.InlineKeyboardButton('Просмотреть .xlsx в браузере')
    markup.add(bttn_1, bttn_2, bttn_3, bttn_4)
    bot.send_message(message.chat.id, 'Парсинг выполнен!', reply_markup = markup)

    bot.register_next_step_handler(message, on_click)

  elif message.text == 'Скачать .xlsx':

    file = open('/content/drive/MyDrive/АД и МО/Lab_5_6/Гитар-гитар.xlsx', 'rb')
    bot.send_document(message.chat.id, file)
    bot.register_next_step_handler(message, on_click)

  elif message.text == 'Просмотреть .xlsx в браузере':

    bot.send_message(message.chat.id, 'https://docs.google.com/spreadsheets/d/1b53Iz0I0IDsR9UYK_G6DJmMMSl12miXS/edit?usp=drive_link&ouid=115227375262358642455&rtpof=true&sd=true')
    bot.register_next_step_handler(message, on_click)

  elif message.text == '/start':

    start(message)

  elif message.text == '/help':

    help(message)

  else:

    bot.send_message(message.chat.id, 'Извините, я вас не понимаю, пожалуйста, используйте кнопки!')
    bot.register_next_step_handler(message, on_click)


@bot.message_handler(commands=['start'])
def start(message):

  del_bttn = telebot.types.ReplyKeyboardRemove()
  keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
  bttn_1 = types.InlineKeyboardButton('Перейти на сайт')
  bttn_2 = types.InlineKeyboardButton('Выполнить парсинг')
  keyboard.add(bttn_1, bttn_2)
  bot.send_message(message.chat.id, f'Здравствуйте, {message.from_user.first_name}', reply_markup = del_bttn)
  sleep(1)
  bot.send_message(message.chat.id, 'Советую ознакомиться с интрукцией! Для этого загляните в меню', reply_markup=keyboard)
  bot.register_next_step_handler(message, on_click)

@bot.message_handler(commands=['help'])
def help(message):

  bot.send_message(message.chat.id, 'Благодаря алгоритму парсинга система собирает данные о моделях студийных конденсаторных микрофонов, представленных на сайте магазина музыкальных инструментов "Гитар-Гитар"')
  sleep(3)
  bot.send_message(message.chat.id, 'Затем система помещает данные в .xlsx-файл, который вы можете просмотреть в браузере или скачать')
  bot.register_next_step_handler(message, on_click)

bot.polling(none_stop = True)